In [9]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

# Importing required Libraries for Plotting the Map

import geopandas as gpd
import json
import altair as alt
import pandas as pd

from os import listdir
from os.path import isfile, join
import glob

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import seaborn as sns

print("Third Upload Completed!!")

Repositories uploaded!!
Second Upload Completed!!
Third Upload Completed!!


In [10]:
from dask.distributed import Client
client = Client()

/Users/shrey/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60627 instead
  warnings.warn(


In [11]:
import joblib

########################################## Shorter Individual Datasets (US) ###############################################################

US_df_train = pd.read_csv('US_data/US_10_slice/Train/Train_3.csv')
US_train_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'Latitude', 'Longitude', 'year', 'month', 'rid', 'clust']
US_df_train = US_df_train.drop(US_train_droplist, axis = 1)
US_df_train = US_df_train.rename(columns={"is": "ISS"})
print(US_df_train.columns)

US_df_transfer = pd.read_csv('US_data/US_10_slice/Transfer/Transfer_3.csv')
# 'Latitude', 'Longitude',
US_transfer_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'year', 'month', 'rid', 'clust']
US_df_transfer = US_df_transfer.drop(US_transfer_droplist, axis = 1)
US_df_transfer = US_df_transfer.rename(columns={"is": "ISS"})
print(US_df_transfer.shape)

US_df_test = pd.read_csv('US_data/US_10_slice/Test/Test_3.csv')
# 'Latitude', 'Longitude',
US_test_droplist = ['cmaq_id', 'cmaq_x', 'cmaq_y', 'year', 'month', 'rid']
US_df_test = US_df_test.drop(US_test_droplist, axis = 1)
US_df_test = US_df_test.rename(columns={"is": "ISS"})
print(US_df_test.shape)

target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()

# ####################### STrAdaBoost.R2 #########################################################
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

# # sample_size = [166, 800]
# sample_size = [len(US_df_train_X), len(US_df_transfer_X)]
# n_estimators = 100
# steps = 30
# fold = 10
# random_state = np.random.RandomState(1)

# print("STrAdaBoost.R2 AS")
# print("-------------------------------------------")

# r2scorelist_stradaboost_us = []
# rmselist_stradaboost_us = []

# for x in range(0, 2):

#     model_stradaboost_us = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 4),
#                           n_estimators = n_estimators, sample_size = sample_size,
#                           steps = steps, fold = fold,
#                           random_state = random_state)
    
#     with joblib.parallel_backend('dask'):
#         model_stradaboost_us.fit(np_TF_train_X, np_TF_train_y_list)
#         y_pred_us = model_stradaboost_us.predict(np_US_df_test_X)

#     mse_stradaboost_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_us))
#     rmselist_stradaboost_us.append(mse_stradaboost_us)

#     r2_score_stradaboost_us = pearsonr(np_US_df_test_y_list, y_pred_us)
#     r2_score_stradaboost_us = (r2_score_stradaboost_us[0])**2
#     r2scorelist_stradaboost_us.append(r2_score_stradaboost_us)
    
    
# print("RMSE of STrAdaboost:", statistics.mean(rmselist_stradaboost_us))
# print("R^2 of STrAdaboost:", statistics.mean(r2scorelist_stradaboost_us))
# print("\n")
# print("RMSE of STrAdaboost:", rmselist_stradaboost_us)
# print("R^2 of STrAdaboost:", r2scorelist_stradaboost_us)

# print("-------------------------------------------")

####################### GBR #########################################################
print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBRTL_us = []
rmselist_GBRTL_us = []

for x in range(0, 10):

    model_GBRTL_us = GradientBoostingRegressor(learning_rate=0.01, max_depth = 4, n_estimators=1000, subsample=0.5)
    
    with joblib.parallel_backend('dask'):
        model_GBRTL_us.fit(np_TF_train_X, np_TF_train_y_list)
        y_pred_GBRTL_us = model_GBRTL_us.predict(np_US_df_test_X) 

    rmse_GBRTL_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_GBRTL_us))
    rmselist_GBRTL_us.append(rmse_GBRTL_us)
    
    r2_score_GBRTL_us = pearsonr(np_US_df_test_y_list, y_pred_GBRTL_us)
    r2_score_GBRTL_us = (r2_score_GBRTL_us[0])**2
    r2scorelist_GBRTL_us.append(r2_score_GBRTL_us)
    
print("RMSE of GBR:", statistics.mean(rmselist_GBRTL_us))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBRTL_us))
print("\n")
print("RMSE of GBR:", rmselist_GBRTL_us)
print("R^2 of GBR:", r2scorelist_GBRTL_us)

print("-------------------------------------------")

Index(['pm25_value', 'narr_dpt', 'narr_vis', 'nldas_pevapsfc',
       'nldas_dlwrfsfc', 'nldas_dswrfsfc', 'nldas_cape', 'nldas_pressfc',
       'nldas_tmp2m', 'nldas_rh2m', 'nldas_ugrd10m', 'nldas_vgrd10m',
       'forest_cover', 'elev', 'emissi11', 'local', 'ISS', 'pd', 'gc_aod'],
      dtype='object')
(2856, 21)
(1310, 21)
Gradient Boosting Regression
-------------------------------------------


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [67]:
######################################## Phase 1: Seeding Technique (US) ###################################################

kmeans = KMeans(n_clusters = 15, random_state=0).fit(US_df_transfer)

US_alternate_df = US_df_transfer.copy()
US_alternate_df_np = US_df_transfer.to_numpy()

idxlist = []
US_new_df_list = []

for rowkm in kmeans.cluster_centers_:
    mindist = -99
    rowidx = 0
    idx = 0
    for row in US_alternate_df_np:
        dst = distance.euclidean(row, rowkm)

        if(dst >= mindist):
            mindist = dst
            rowidx = idx
            rowval = row

        idx = idx + 1

#     print("Row selected: ", rowidx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", mindist)
#     print("Matrix shape: ", kinematics_alternate_df_np.shape)
    US_new_df_list.append(rowval)
    US_alternate_df = np.delete(US_alternate_df_np, rowidx, 0)
    idxlist.append(rowidx)


US_new_df = pd.DataFrame(np.vstack(US_new_df_list))

print("Shape of dataset extracted: ")
print(US_new_df.shape)
print("----------------------------------------------")

Shape of dataset extracted: 
(15, 21)
----------------------------------------------


In [68]:
##################################################### Phase 2: Seeding (US) ################################################

US_alternate_transfer_df = US_df_transfer[1:].copy()
US_alternate_transfer_df_np = US_alternate_transfer_df.to_numpy()

idxlist2 = []
US_final_df_list = []

for row_nm in US_new_df_list:
    min_dist = -99
    row_idx = 0
    idx_val = 0
    for row_alt in US_alternate_transfer_df_np:
        dst = distance.euclidean(row_alt, row_nm)
        if(dst >= mindist):
            min_dist = dst
            row_idx = idx_val
            row_val = row_alt

        idx_val = idx_val + 1

#     print("Row selected: ", row_idx) #Alternate_df.loc[rowidx,:]\
#     print("Min. distance: ", min_dist)
#     print("Matrix shape: ", Elevators_alternate_source_df_np.shape)
    US_final_df_list.append(row_val)
    US_alternate_transfer_df_np = np.delete(US_alternate_transfer_df_np, row_idx, 0)
    idxlist2.append(row_idx)


US_final_df = pd.DataFrame(np.vstack(US_final_df_list), columns = US_df_transfer.columns)

print("----------------------------------------------")
print("Shape of source before :",US_df_transfer.shape)
US_df_transfer = pd.DataFrame(np.vstack(US_alternate_transfer_df_np), columns= US_df_transfer.columns)
print("Shape of source after :", US_df_transfer.shape)

print("----------------------------------------------")
print("Shape of target before :", US_df_train.shape)
US_df_train = pd.concat([US_df_train, US_final_df], ignore_index=True)
print("Shape of target after :", US_df_train.shape)

print("----------------------------------------------")

----------------------------------------------
Shape of source before : (3173, 21)
Shape of source after : (3157, 21)
----------------------------------------------
Shape of target before : (151, 19)
Shape of target after : (166, 21)
----------------------------------------------


In [69]:
################################## Finding best instances from the source dataset (US) ######################################################

US_df_transfer["ManDis"] = ""

US_df_train_mean = []
prow = US_df_train.mean()
US_df_train_mean = [prow.pm25_value, prow.narr_dpt, prow.narr_vis, prow.nldas_pevapsfc, prow.nldas_dlwrfsfc, prow.nldas_dswrfsfc, 
                          prow.nldas_cape, prow.nldas_pressfc, prow.nldas_tmp2m, prow.nldas_rh2m, prow.nldas_ugrd10m, prow.nldas_vgrd10m,
                          prow.forest_cover, prow.elev, prow.emissi11, prow.local, prow.ISS, prow.pd, prow.gc_aod]

rowidx = 0

for row in US_df_transfer.itertuples():
    row_list =[row.pm25_value, row.narr_dpt, row.narr_vis, row.nldas_pevapsfc, row.nldas_dlwrfsfc, row.nldas_dswrfsfc, 
            row.nldas_cape, row.nldas_pressfc, row.nldas_tmp2m, row.nldas_rh2m, row.nldas_ugrd10m, row.nldas_vgrd10m,
            row.forest_cover, row.elev, row.emissi11, row.local, row.ISS, row.pd, row.gc_aod]

    man_dis = 0
    for i in range(0, len(row_list)):
        tempval = US_df_train_mean[i] - row_list[i]
        man_dis = man_dis + abs(tempval)

    US_df_transfer.loc[rowidx,"ManDis"] = man_dis
    rowidx = rowidx + 1

US_df_transfer = US_df_transfer.sort_values(by =['ManDis'])
US_df_transfer = US_df_transfer.head(800) 
US_df_transfer = US_df_transfer.drop(['ManDis'], axis =1)
US_df_transfer = US_df_transfer.reset_index(drop=True)

#################### Splitting with small target set and large source and test set #############
print("Target Set: ", US_df_train.shape)
print("Source Set: ", US_df_transfer.shape)
print("Test Set: ", US_df_test.shape)


target_column = ['pm25_value']
US_df_train_y = US_df_train[target_column]
US_df_train_X = US_df_train.drop(target_column, axis = 1)

US_df_transfer_y = US_df_transfer[target_column]
US_df_transfer_X = US_df_transfer.drop(target_column, axis = 1)

US_df_test_y = US_df_test[target_column]
US_df_test_X = US_df_test.drop(target_column, axis = 1)

TF_train_X = pd.concat([US_df_transfer_X, US_df_train_X], sort= False)
TF_train_y = pd.concat([US_df_transfer_y, US_df_train_y], sort= False)

np_TF_train_X = TF_train_X.to_numpy()
np_TF_train_y = TF_train_y.to_numpy()

# np_TF_train_X = US_df_train_X.to_numpy()
# np_TF_train_y = US_df_train_y.to_numpy()

np_US_df_test_X = US_df_test_X.to_numpy()
np_US_df_test_y = US_df_test_y.to_numpy()

np_TF_train_y_list = np_TF_train_y.ravel()
np_US_df_test_y_list = np_US_df_test_y.ravel()

print("---------------------------")

###############################################################################################################################################

Target Set:  (166, 21)
Source Set:  (800, 21)
Test Set:  (1312, 21)
---------------------------


In [15]:
####################### GBR #########################################################
print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBRTL_us = []
rmselist_GBRTL_us = []

for x in range(0, 50):

    model_GBRTL_us = GradientBoostingRegressor(learning_rate=0.01, max_depth = 4, n_estimators=1000, subsample=0.5)
    model_GBRTL_us.fit(np_TF_train_X, np_TF_train_y_list)


    y_pred_GBRTL_us = model_GBRTL_us.predict(np_US_df_test_X) 

    rmse_GBRTL_us = sqrt(mean_squared_error(np_US_df_test_y_list, y_pred_GBRTL_us))
    rmselist_GBRTL_us.append(rmse_GBRTL_us)
    
    r2_score_GBRTL_us = pearsonr(np_US_df_test_y_list, y_pred_GBRTL_us)
    r2_score_GBRTL_us = (r2_score_GBRTL_us[0])**2
    r2scorelist_GBRTL_us.append(r2_score_GBRTL_us)
    
print("RMSE of GBR:", statistics.mean(rmselist_GBRTL_us))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBRTL_us))
print("\n")
print("RMSE of GBR:", rmselist_GBRTL_us)
print("R^2 of GBR:", r2scorelist_GBRTL_us)

print("-------------------------------------------")

Gradient Boosting Regression
-------------------------------------------
RMSE of GBR: 4.783353178405265
R^2 of GBR: 0.3917969715012284


RMSE of GBR: [4.777167680246299, 4.776656488331672, 4.792167633971575, 4.77982172902257, 4.819662721441523, 4.805859486893886, 4.831131502066503, 4.759630307946516, 4.79899446865215, 4.808078307972538, 4.7689097926498025, 4.761545761751902, 4.793867387103472, 4.8147914093882695, 4.7655625937951696, 4.765246636291876, 4.779461469008211, 4.761944550174594, 4.791395899000361, 4.769362149103635, 4.765730449509007, 4.811249776692141, 4.779585665372544, 4.743549312680322, 4.780654938545946, 4.7927558303344835, 4.785782876923928, 4.810800610707273, 4.772524942200618, 4.745572891314459, 4.734878251732028, 4.804620211382785, 4.780356293136857, 4.790355004233136, 4.77102131260422, 4.812287978346956, 4.788217867993365, 4.8012932096556415, 4.765265950503049, 4.721781386814165, 4.737439532293898, 4.810729928548911, 4.751732121372508, 4.80832855705341, 4.8164623010